In [ ]:


from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("MovieRecommendation") \
    .getOrCreate()

# Load the ratings data from CSV
ratings_data = spark.read.csv("ratings.csv", header=True, inferSchema=True)

# Load the movies data from CSV
movies_data = spark.read.csv("movies.csv", header=True, inferSchema=True)

# Split the data into training and test sets
(training_data, test_data) = ratings_data.randomSplit([0.8, 0.2])

# Create an ALS model
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

# Fit the model to the training data
model = als.fit(training_data)

# Generate recommendations for all users
user_recs = model.recommendForAllUsers(10)  # Generate top 10 recommendations for each user

# Show the recommendations for a specific user
user_id = 1
user_rec = user_recs.filter(user_recs.userId == user_id).select("recommendations")
user_rec.show(truncate=False)

# Show the movies rated by the user
user_ratings = ratings_data.filter(ratings_data.userId == user_id).join(movies_data, "movieId")
user_ratings.show(truncate=False)
